### Домашнее задание

#### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class




In [1]:
import pandas as pd

In [2]:
movies =  pd.read_csv("./ml-latest-small/movies.csv")
ratings = pd.read_csv("./ml-latest-small/ratings.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
def rate_movies(rate):
    if rate   <= 2.0 :  return "low"
    elif rate <= 4.5 :  return "medium"
    elif rate >= 4.5 :  return "high"
    else             :  return None

In [5]:
ratings["class"] = ratings["rating"].apply(rate_movies)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,medium
1,1,3,4.0,964981247,medium
2,1,6,4.0,964982224,medium
3,1,47,5.0,964983815,high
4,1,50,5.0,964982931,high


#### Задание 2
Используем файл __keywords.csv__.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону.

Т. е. если поисковый запрос содержит название города региона, то в столбце __‘region’__ пишется название этого региона.
Если поисковый запрос не содержит названия города, то ставим __‘undefined’__.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [6]:
geo_data = {

            'Центр': ['москва', 'тула', 'ярославль'],
            'Северо-Запад': ['петербург', 'псков', 'мурманск'],
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }

Результат классификации запишите в отдельный столбец __region__.

In [7]:
keywords = pd.read_csv("keywords.csv")
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [8]:
def classify_region(keyword):
    for region,cities in geo_data.items():
        for city in cities:
            if city.upper() in keyword.upper():
                return region
    return "undefined"

In [9]:
keywords["region"] = keywords["keyword"].apply(classify_region)

In [10]:
keywords[ keywords["region"] != "undefined" ].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


In [11]:
keywords[ keywords["region"] == "undefined" ].head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.


При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:  
В переменную __years__ запишите список из всех годов с 1950 по 2010.  
Напишите функцию *production_year*, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

 - для каждой строки пройдите по всем годам списка __years__
 - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
 - если ни один из номеров года списка __years__ не встретился в названии фильма, то возвращается 1900 год 
 - Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец __‘year’__  

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [12]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [13]:
years = list(range(1950,2010))
def production_year(film_title):
    for year in years:
        if str(year) in film_title:
            return year
    return 1900

movies['year'] = movies['title'].apply(production_year)

Т.к. данных по ср. рейтингу фильма в movies.csv нет - предварительно посчитаем средний рейтинг всех фильмов.

In [14]:
# Заранее агрегируем данные, чтобы не считать при каждой итерации
movieRatings = ratings.groupby("movieId")["rating"].mean()
movieRatings.head()

movieId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: rating, dtype: float64

Внесем данные по среднему рейтингу каждого фильма

In [15]:
# Series.get(self, key[, default])	Get item from object for given key (ex: DataFrame column).
def avg_rating(movieId):
    rating = movieRatings.get(movieId,None)
    if rating:
        return float(rating)

# Сначала хотел сделать так:
# movies['avg_rating'] = movies['movieId'].apply(movieRatings.get,None) 
# Но type movies['avg_rating'] =  Object а не Float - не смог справиться =(

movies['avg_rating'] = movies['movieId'].apply(avg_rating)
movies.head()

,movieId,title,genres,year,avg_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.357143
4,5,Father of the Bride Part II (1995),Comedy,1995,3.071429


Сгруппируем по годам по среднему рейтингу, отсортируем и выведем

In [16]:
movies.groupby("year").agg({"avg_rating":"mean"}).sort_values(by="avg_rating", ascending = False)

,avg_rating
year,
1971,3.811243
1957,3.767680
1951,3.718044
1952,3.700104
1955,3.672863
1958,3.668078
1976,3.658863
1961,3.656946
1972,3.645005
